In [1]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.3.2
!pip install  -U autogluon

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 88.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.3.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (s

In [2]:
# Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv
/kaggle/input/calories-burnt-prediction/calories.csv


In [4]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
display(train_df.head())
display(train_df.shape)

# In the sidebar include the Calories Burnt Prediction dataset to access this.
orig_df = pd.read_csv('/kaggle/input/calories-burnt-prediction/calories.csv')
orig_df.rename(columns={'User_ID':'id','Gender':'Sex'}, inplace=True)
display(orig_df.head())
display(orig_df.shape)

# Combine the two datasets
train_df = pd.concat([train_df, orig_df], axis=0, ignore_index=True)
train_df = train_df.drop_duplicates()
display(train_df.shape)

# Check for missing values
display(train_df.isnull().sum())

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


(750000, 9)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,14733363,male,68,190.0,94.0,29.0,105.0,40.8,231.0
1,14861698,female,20,166.0,60.0,14.0,94.0,40.3,66.0
2,11179863,male,69,179.0,79.0,5.0,88.0,38.7,26.0
3,16180408,female,34,179.0,71.0,13.0,100.0,40.5,71.0
4,17771927,female,27,154.0,58.0,10.0,81.0,39.8,35.0


(15000, 9)

(765000, 9)

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [5]:
def rmsle(y_true, y_pred):
    # Ensure predictions are non-negative
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [6]:
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularPredictor

rmsle_scorer = make_scorer(
    name='rmsle',
    score_func=rmsle,
    optimum=0,  # Lower is better for RMSLE
    greater_is_better=False,
    needs_proba=False
)

PREDICTOR_AUTO = TabularPredictor(
                                  label='Calories', 
                                  eval_metric=rmsle_scorer,
                                  problem_type="regression"
                                 )
# Fitting Predictor
PREDICTOR_AUTO.fit(train_df,presets='best_quality',
                 time_limit=3600 * 4,
                 keep_only_best=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20250506_142115"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.07 GB / 31.35 GB (95.9%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be 

In [7]:
PREDICTOR_AUTO.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.061743,rmsle,159.580148,2357.875084,0.021878,3.290310,3,True,8
1,ExtraTreesMSE_BAG_L2,-0.061795,rmsle,126.069144,994.864197,29.144089,249.613111,2,True,7
2,RandomForestMSE_BAG_L2,-0.062325,rmsle,130.414182,2104.971663,33.489126,1359.720577,2,True,6
3,ExtraTreesMSE_BAG_L1,-0.064838,rmsle,30.158388,164.731761,30.158388,164.731761,1,True,4
4,RandomForestMSE_BAG_L1,-0.065174,rmsle,29.150142,435.167383,29.150142,435.167383,1,True,3
5,ExtraTrees_r42_BAG_L1,-0.065681,rmsle,31.615227,142.680076,31.615227,142.680076,1,True,5
6,KNeighborsDist_BAG_L1,-0.590038,rmsle,3.043321,1.004030,3.043321,1.004030,1,True,2
7,KNeighborsUnif_BAG_L1,-0.608045,rmsle,2.957977,1.667836,2.957977,1.667836,1,True,1


In [8]:
test_df = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')
y_test = PREDICTOR_AUTO.predict(test_df)

2025-05-06 18:22:35,546	ERROR worker.py:420 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-05-06 18:22:35,548	ERROR worker.py:420 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2025-05-06 18:22:35,549	ERROR worker.py:420 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: Futur

In [9]:
"""train_df['Sex'] = LabelEncoder().fit_transform(train_df['Sex'])
# Convert sex to 0 and 1
#train_df['Sex'].replace({'male':0, 'female': 1}, inplace=True)


#test_df['Sex'].replace({'male':0, 'female': 1}, inplace=True)
test_df['Sex'] = LabelEncoder().fit_transform(test_df['Sex'])
display(test_df.head())
display(test_df.isnull().sum())

# Find which of these are correlated with the calories
plt.figure(figsize = (12,8))
sns.heatmap(train_df.corr(), cmap='coolwarm')
plt.show()
"""

"train_df['Sex'] = LabelEncoder().fit_transform(train_df['Sex'])\n# Convert sex to 0 and 1\n#train_df['Sex'].replace({'male':0, 'female': 1}, inplace=True)\n\n\n#test_df['Sex'].replace({'male':0, 'female': 1}, inplace=True)\ntest_df['Sex'] = LabelEncoder().fit_transform(test_df['Sex'])\ndisplay(test_df.head())\ndisplay(test_df.isnull().sum())\n\n# Find which of these are correlated with the calories\nplt.figure(figsize = (12,8))\nsns.heatmap(train_df.corr(), cmap='coolwarm')\nplt.show()\n"

In [10]:
"""
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X,y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

scores = make_mi_scores(X,y)
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(scores)
"""

'\nfrom sklearn.feature_selection import mutual_info_regression\ndef make_mi_scores(X, y):\n    mi_scores = mutual_info_regression(X,y)\n    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)\n    mi_scores = mi_scores.sort_values(ascending=False)\n    return mi_scores\n\ndef plot_mi_scores(scores):\n    scores = scores.sort_values(ascending=True)\n    width = np.arange(len(scores))\n    ticks = list(scores.index)\n    plt.barh(width, scores)\n    plt.yticks(width, ticks)\n    plt.title("Mutual Information Scores")\n\nscores = make_mi_scores(X,y)\nplt.figure(dpi=100, figsize=(8, 5))\nplot_mi_scores(scores)\n'

In [11]:

"""
# Feature Augmentation
for df in [train_df, test_df]:
    df['BMI'] = df['Weight']/((df['Height'] /100) **2)
    df['Intensity'] = df['Duration'] * df['Heart_Rate']
    #df['Sex'] = df['Sex'].astype('category')

# Remove the columns
X = train_df.drop(columns=['Calories', 'id'])
y = train_df['Calories']
X_test = test_df.drop(columns=['id'])

# Plot each variable
features = train_df.columns.tolist()
for feature in features:
    plt.figure(figsize=(6, 4))
    sns.histplot(train_df[feature], kde=True)
    plt.title(f'Distribution of {feature}')
    plt.show()
"""

"\n# Feature Augmentation\nfor df in [train_df, test_df]:\n    df['BMI'] = df['Weight']/((df['Height'] /100) **2)\n    df['Intensity'] = df['Duration'] * df['Heart_Rate']\n    #df['Sex'] = df['Sex'].astype('category')\n\n# Remove the columns\nX = train_df.drop(columns=['Calories', 'id'])\ny = train_df['Calories']\nX_test = test_df.drop(columns=['id'])\n\n# Plot each variable\nfeatures = train_df.columns.tolist()\nfor feature in features:\n    plt.figure(figsize=(6, 4))\n    sns.histplot(train_df[feature], kde=True)\n    plt.title(f'Distribution of {feature}')\n    plt.show()\n"

In [12]:
"""
# Scale the features
scalar = StandardScaler()
X_scaled = scalar.fit_transform(X)
X_test = scalar.fit_transform(X_test)
X_scaled
"""

'\n# Scale the features\nscalar = StandardScaler()\nX_scaled = scalar.fit_transform(X)\nX_test = scalar.fit_transform(X_test)\nX_scaled\n'

In [13]:
"""
# Split train and validation datasets
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=36
)

# Create a Stacking Regressor
base_models = [
    ('random_forest', RandomForestRegressor(n_estimators=50, max_depth=12, random_state=36)),
    ('xgboost', XGBRegressor(random_state=36, max_depth=12, learning_rate=0.02, eval_metric='logloss')),
    ('linear', LinearRegression()),
    ('svr', SVR(kernel='rbf', C=1.0, epsilon=0.1)),
    ('gbr', GradientBoostingRegressor(n_estimators=50, learning_rate=0.05, max_depth=6, random_state=36))
]
meta_model = RidgeCV()

best_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5  # Cross-validation folds for out-of-fold predictions
)
best_model.fit(X_train, y_train)
# Validation metrics
y_pred = best_model.predict(X_val)
"""

"\n# Split train and validation datasets\nX_train, X_val, y_train, y_val = train_test_split(\n    X_scaled, y, \n    test_size=0.2, \n    random_state=36\n)\n\n# Create a Stacking Regressor\nbase_models = [\n    ('random_forest', RandomForestRegressor(n_estimators=50, max_depth=12, random_state=36)),\n    ('xgboost', XGBRegressor(random_state=36, max_depth=12, learning_rate=0.02, eval_metric='logloss')),\n    ('linear', LinearRegression()),\n    ('svr', SVR(kernel='rbf', C=1.0, epsilon=0.1)),\n    ('gbr', GradientBoostingRegressor(n_estimators=50, learning_rate=0.05, max_depth=6, random_state=36))\n]\nmeta_model = RidgeCV()\n\nbest_model = StackingRegressor(\n    estimators=base_models,\n    final_estimator=meta_model,\n    cv=5  # Cross-validation folds for out-of-fold predictions\n)\nbest_model.fit(X_train, y_train)\n# Validation metrics\ny_pred = best_model.predict(X_val)\n"

In [14]:
"""
error = rmsle(y_val, y_pred)
print(f'RMSLE on Validation Set: {error}')
"""

"\nerror = rmsle(y_val, y_pred)\nprint(f'RMSLE on Validation Set: {error}')\n"

In [15]:
"""
importances = best_model.feature_importances_
features = X_train.columns.tolist()
sns.barplot(x=importances, y=features)
plt.title('Feature Importances')
plt.show()
"""

"\nimportances = best_model.feature_importances_\nfeatures = X_train.columns.tolist()\nsns.barplot(x=importances, y=features)\nplt.title('Feature Importances')\nplt.show()\n"

In [16]:
# 1 - Fast
# RandomForestRegressor(n_estimators=50, max_depth=2, random_state=36)
# XGBRegressor(random_state=36, max_depth=2, learning_rate=0.02, eval_metric='logloss') 
# Mean Absolute Error on Validation Set: 7.8085299589503

# 2 - Medium
# RandomForestRegressor(n_estimators=50, max_depth=2, random_state=36)
# XGBRegressor(subsample = 0.8252835138864835, reg_lambda = 3.501605991491236,
#                             reg_alpha= 9.019718559862135, n_estimators= 494, max_depth= 12,
#                             learning_rate= 0.025720450946032463, gamma= 1.8903496577328514,
#                             colsample_bytree= 0.5789286073361388, min_child_weight =  7)
# Mean Absolute Error on Validation Set: 2.1471820120331357

# 3 - Very slow
# RandomForestRegressor(n_estimators=100, max_depth=10, random_state=36)
# XGBRegressor(n_estimators= 400, max_depth= 12, random_state=36)
# Mean Absolute Error on Validation Set: 2.311597161890868

# 4 - 16 mins
# RandomForestRegressor(n_estimators=100, max_depth=10, random_state=36)),
# XGBRegressor(subsample = 0.8252835138864835, reg_lambda = 3.501605991491236,
#                             reg_alpha= 9.019718559862135, n_estimators= 494, max_depth= 12,
#                             learning_rate= 0.025720450946032463, gamma= 1.8903496577328514,
#                             colsample_bytree= 0.5789286073361388, min_child_weight =  7))
# 2.145, Leaderboard = 0.128

# 5 - 9 mins
# RandomForestRegressor(n_estimators=50, max_depth=12, random_state=36)
# XGBRegressor(random_state=36, max_depth=12, learning_rate=0.02, eval_metric='logloss')
# 2.217, Leaderboard = 0.126

# 6 
# Same as above, included intensity and BMI as features
# 2.204

# 7
# included LinearRegressor, SVR + GBR as well and extra features

In [17]:
"""
# Evaluate in test and submit
y_test = best_model.predict(X_test)
"""

'\n# Evaluate in test and submit\ny_test = best_model.predict(X_test)\n'

In [18]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "Calories": y_test
})

submission.to_csv("submission.csv", index=False)
